## Lab 11: Predicting Low Birthweight 

Please complete this lab by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

This assignment is due by **11:59pm on Thursday, April 28**.

In [ ]:
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# This is for linear regression
from sklearn.linear_model import LinearRegression

# This is for logistic regression
from sklearn.linear_model import LogisticRegression

# For getting results from the test set
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

### North Carolina Births Data

In this lab, we will work with a dataset of births in North Carolina. A series of variables were collected, most notably the smoking habit of the mother as well as the birthweight of the baby. We are interested in what factors are associated with a low birthweight. We'll first look at predicting `birthweight` as a numerical variable, then use the categorical `lowbirthweight` and try to predict low birthweight status using logistic regression.

In [ ]:
ncbirths = Table.read_table('ncbirths.csv')

In [ ]:
ncbirths.show(5)

Note that we need to work with the data a little bit to be able to fit a model using categorical data. The `sklearn` package only allows for categorical variables that have been changed into dummy variables (that is, 0/1 variables). So, we're going to need to create new variables that contain the same information, except as numbers. Luckily, True/False maps onto 1/0, so we can just do comparisons. We'll use a lot of these variables later, so let's do some cleaning now.

In [ ]:
ncbirths_dummy = ncbirths.with_columns('premature', ncbirths.column('premie') == 'premie', # True if premature, False if not
                     'female', ncbirths.column('gender') == 'female', # True if female, False if not
                     'smoker', ncbirths.column('habit') == 'smoker', # True if smoker, False if not
                     'label', ncbirths.column('lowbirthweight') == 'low') # Our outcome. True if low birthweight, False if not


ncbirths_dummy.show(5)

In [ ]:
# Drop redundant rows now
ncbirths_with_dummies = ncbirths_dummy.drop('premie', 'gender', 'habit','lowbirthweight')

<font color = 'red'>**Question 1. Using `ncbirths_with_dummies`, change the `whitemom` and `mature` variables so that they are True/False values. Add a column called `younger` which is True if `mature` is "younger mom" and False otherwise, and a column called `white` which is True if `whitemom` is "white" and False otherwise. Assign this to `ncbirths_clean`. Make sure to remove the redundant variables (`whitemom` and `mature`).**</font>

Now that we've cleaned up our dataset, let's try fitting a logistic regression line.

## Using a Logistic Regression

A logistic regression is essentially the same as a linear regression, except we have a binary (two category) categorical variable instead of a numerical variable as the outcome, and we get predicted probabilities out of the predictions. The way we fit a logistic regression model is essentially the same as a linear regression model with `sklearn`. We set up a `LogisticRegression` object first, then we use `fit` to provide it the data. 

In [ ]:
# Create a model object
logit = LogisticRegression()

# Define predictor and outcome
predictor = ncbirths_clean.select('mage', 'weeks', 'female', 'smoker').rows
outcome = ncbirths_clean.column('label')

# Fit the model
logit.fit(X = predictor, y = outcome)

We can get the coefficients and intercept the same way as well.

In [ ]:
logit.coef_

In [ ]:
logit.intercept_

<font color = 'red'>**Question 2. Write out the form of the equation in the logistic regression model that we just ran. You can just use `logit(birthweight status)` as the outcome instead of writing out the logit transformation.**</font>

*Your answer here.*

<font color = 'red'>**Question 3. Focusing on just the sign of the coefficients, interpret the relationship between each of the predictors and whether the baby had a low birthgweight.**</font>

*Your answer here.*

## Intuition for Logistic Regression

We are treating the outcome variable (in this case, low birthweight status) as a 0/1 variable. The graph below shows the relationship between number of weeks of pregnancy and the actual outcome. 

In [ ]:
weeks_and_weight = Table().with_columns('Weeks', ncbirths_clean.column('weeks'),
                                       'Low Birthweight Status', ncbirths_clean.column('label').astype(int))
weeks_and_weight.scatter('Weeks', 'Low Birthweight Status')

Now let's add in gold the predicted probabilities at the different number of weeks. 

In [ ]:
weeks_and_weight = Table().with_columns('Weeks', ncbirths_clean.column('weeks'),
                                       'Low Birthweight Status', ncbirths_clean.column('label').astype(int))
weeks_and_weight.scatter('Weeks', 'Low Birthweight Status')
plt.scatter(ncbirths_clean.column('weeks'), logit.predict_proba(predictor)[:,1], lw=1, color='gold')

The logit transformation makes it so that even though there is a linear relationship between the (transformed) outcome and the predictors, all predicted probabilities are between 0 and 1, and the relationship between weeks and low birthweight status can clearly be seen here, with higher number of weeks corresponding to lower probability of low birthweight.

<font color = 'red'>**Question 4. Try adding another predictor to the logistic regression model and running it again. Make sure you do not use `weight`, since that is exactly what the `lowbirthweight` variable is based on. How does your model change? What are the coefficients? How would you interpret the coefficients for the variable you added?**</font>

### Predictions using Logistic Regression

To do predictions, we use the `predict_proba` method rather than the `predict` method. This will give us a set of two probabilities for each row. The first represents the probability of a 0, while the second represents the probability of a 1. Here, we are more interested in predicting the probability that a baby will have a low birthweight so that we can provide some sort of help or care in advance. So, we'll just try to find the ones with the highest probability. 

In this example, we simply use the same dataset (`ncbirths`) to make predictions, but in reality, you'd use a different dataset. In the next section, we'll go over something called training and testing data that can help improve your models and their performance on future data. 

In [ ]:
# Predict for each element in original dataset
# Each row has probabilities for 0 and 1
# Should add up to one on each row.
logit.predict_proba(predictor)

In [ ]:
# Let's extract predicted probability of being low birthweight.
# The [:,1] means that we want everything from the rows, and only the column with index 1 (so, the second column)
lowbirthweight_probability = logit.predict_proba(predictor)[:,1]

In [ ]:
Table().with_column('Low Birthweight Predicted Probability',lowbirthweight_probability).hist()

<font color = 'red'>**Question 5. What would be the predicted probability of a baby being lowbirthweight according to our model if the mother's age was 30, the pregnancy lasted 36 weeks, the mother was not a smoker, and the baby was female?**</font>

**Hint:** You'll need to create a Table to use with `predict_proba`.

## Splitting into Train and Test

What if we wanted to get a sense for how our model might perform when given new data? We can split our data into train and test sets, build our model using the train, and assess our model using test.

First, let's start by taking a sample of our data. We'll use 80% of the data to build our model, then using the remaining 20% to see how it is doing. To do this, we first shuffle our dataset (the cleaned version), then take the first 80% of the rows as `train`. Then, we assign the rest to `test`. The shuffling is to make sure that we are doing a random split.

In [ ]:
shuffled_births = ncbirths_clean.sample(with_replacement = False)

nrows = ncbirths.num_rows
ntrain = int(nrows*.8) # 80% used to train

train = shuffled_births.take(np.arange(ntrain))
test = shuffled_births.take(np.arange(ntrain,nrows))

In [ ]:
train.num_rows

In [ ]:
test.num_rows

Now, we can take the same exact steps as we did earlier, except using the `train` object to build our model.

<font color = 'red'>**Question 6. Fit a logistic regression model using `train` with `mage`, `weeks`, `female`, and `smoker` as the predictors. What are the coefficients? How does it differ from fitting the model using the full data?**</font>

Next, we can use `predict` to predict for the cases in `test` and see how our model did. The `predict` function takes the predicted probabilities and assigns a value of 1 as the prediction if the probability is above 0.5 and 0 if it is below 0.5.

In [ ]:
test_predictions = logit.predict(test.select('mage', 'weeks', 'female', 'smoker').rows)
confusion_matrix(test.column('label'), test_predictions)

The columns represent predictions and the rows represent actual values, so the top left is true negatives, the bottom right is true positives, the top right is false positives, and the bottom left is false negatives. We can calculate the accuracy of our model using the `accuracy_score` function. This takes the diagonal values and divides by the total number of cases to give us how often we were correct. 

In [ ]:
accuracy_score(test.column('label'), test_predictions)

However, remember that `predict` by default uses 0.5 as the cutoff point for predicting 0 or 1. We might actually want a different cutoff. For example, if we think a 10% chance is a high enough risk that we want to make sure we flag the baby as possibly being low birthweight, then we might want to set a cutoff value of 0.1 instead. This would make it so that we are predicting more babies as low birthweight. We can do this using `predict_proba` and setting a cutoff manually.

In [ ]:
test_predictions = logit.predict_proba(test.select('mage', 'weeks', 'female', 'smoker').rows)[:,1] > 0.1
confusion_matrix(test.column('label'), test_predictions)

In [ ]:
accuracy_score(test.column('label'), test_predictions)

You should notice that the true positives go up, but so do the false positives. That is, we are predicting more babies to be low birthweight, which is able to catch more cases, but we also increase the number of times we incorrectly predict a baby is low birthweight. This is a tradeoff -- the lower we set the threshold, the fewer false negatives we get, but also the more false positives as well.

You probably observed a lower accuracy for one of the models. This doesn't necessarily mean we would always prefer the higher accuracy predictions. For example, we might be ok with more errors as long as we are able to catch all of the babies who might be at risk of being low birthweight.

<font color = 'red'>**Question 7. Try a few different values for the cutoff other than 0.1 and 0.5. How does the number of false positives and false negatives change? How does the accuracy change? What do you think would be the preferred set of predictions to use?**</font>